<a href="https://colab.research.google.com/github/SawantDisha/Google-Collab-Projects/blob/main/Gmail_Spam_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np   
import pandas as pd  
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

In [2]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/My Drive/IIT Kanpur/ 

/content/drive/My Drive/IIT Kanpur


In [4]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# 1) Loading Dataset

In [5]:
dt = pd.read_csv("spam.csv", encoding= 'unicode_escape')
dt.head(10)

,type,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [6]:
dt['spam'] = dt['type'].map({'spam': 1, 'ham': 0}).astype(int)
dt.head(5)

,type,text,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [7]:
print("Columns in the given data:")
for col in dt.columns:
  print(col)

Columns in the given data:
type
text
spam


In [8]:
t=len(dt['type'])
print("No. of rows in the review column: ", t)
t=len(dt['text'])
print("No. of rows in the liked column: ", t)

No. of rows in the review column:  116
No. of rows in the liked column:  116


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }\n",
\n",
    .dataframe thead th {
        text-align: right;
    }
</style>
<table border=1 class=\"dataframe\">
  <thead>
    <tr style=\"text-align: right;
      <th></th>
      <th>type</th>
      <th>text</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>ham</td>
      <td>Go until jurong point, crazy.. Available only ...</td>
    </tr>
    <tr>
      <th>1</th>
      <td>ham</td>
      <td>Ok lar... Joking wif u oni...</td>
    </tr>
    <tr>
      <th>2</th>
      <td>spam</td>
      <td>Free entry in 2 a wkly comp to win FA Cup fina...</td>
    </tr>
    <tr>
      <th>3</th>
      <td>ham</td>
      <td>U dun say so early hor... U c already then say...</td>
    </tr>
    <tr>
      <th>4</th>
      <td>ham</td>
      <td>Nah I don't think he goes to usf, he lives aro...</td>
    </tr>
    <tr>
      <th>5</th>
      <td>spam</td>
      <td>FreeMsg Hey there darling it's been 3 week's n...</td>
    </tr>
    <tr>
      <th>6</th>
      <td>ham</td>
      <td>Even my brother is not like to speak with me. ...</td>
    </tr>
    <tr>
      <th>7</th>
      <td>ham</td>
      <td>As per your request 'Melle Melle (Oru Minnamin...</td>
    </tr>
    <tr>
      <th>8</th>
      <td>spam</td>
      <td>WINNER!! As a valued network customer you have...</td>
    </tr>
    <tr>
      <th>9</th>
      <td>spam</td>
      <td>Had your mobile 11 months or more? U R entitle...</td>
    </tr>
  </tbody>
</table>
</div>"

# 2) Tokenization

In [9]:
dt['text'][1] #before

'Ok lar... Joking wif u oni...'

In [10]:
def tokenizer(text):
  return text.split()

In [11]:
dt['text']=dt['text'].apply(tokenizer)

In [12]:
dt['text'][1] #after

['Ok', 'lar...', 'Joking', 'wif', 'u', 'oni...']

# 3) Stemming

In [13]:
dt['text'][1] #before

['Ok', 'lar...', 'Joking', 'wif', 'u', 'oni...']

In [14]:
porter = SnowballStemmer("english", ignore_stopwords=False)

In [15]:
def stem_it(text):
  return [porter.stem(word) for word in text]

In [16]:
dt['text']=dt['text'].apply(stem_it)

In [17]:
dt['text'][1] #after

['ok', 'lar...', 'joke', 'wif', 'u', 'oni...']

# 4) Lemmintization

In [18]:
dt['text'][52] #before

['k',
 'fyi',
 'x',
 'has',
 'a',
 'ride',
 'earli',
 'tomorrow',
 'morn',
 'but',
 'he',
 'crash',
 'at',
 'our',
 'place',
 'tonight']

In [19]:
lemmatizer = WordNetLemmatizer()

In [20]:
def lemmit_it(text):
  return [lemmatizer.lemmatize(word, pos="a") for word in text]

In [21]:
dt['text']=dt['text'].apply(lemmit_it)

In [22]:
dt['text'][52] #after

['k',
 'fyi',
 'x',
 'has',
 'a',
 'ride',
 'earli',
 'tomorrow',
 'morn',
 'but',
 'he',
 'crash',
 'at',
 'our',
 'place',
 'tonight']

# 5) Stopword Removal

In [23]:
dt['text'][25] #before

['just',
 'forc',
 'myself',
 'to',
 'eat',
 'a',
 'slice.',
 "i'm",
 'realli',
 'not',
 'hungri',
 'tho.',
 'this',
 'sucks.',
 'mark',
 'is',
 'get',
 'worried.',
 'he',
 'know',
 "i'm",
 'sick',
 'when',
 'i',
 'turn',
 'down',
 'pizza.',
 'lol']

In [24]:
stop_words = stopwords.words('english')

In [25]:
def stop_it(text):
  review = [word for word in text if not word in stop_words]
  return review

In [26]:
dt['text']=dt['text'].apply(stop_it)

In [27]:
dt['text'][25] #after

['forc',
 'eat',
 'slice.',
 "i'm",
 'realli',
 'hungri',
 'tho.',
 'sucks.',
 'mark',
 'get',
 'worried.',
 'know',
 "i'm",
 'sick',
 'turn',
 'pizza.',
 'lol']

In [28]:
dt.head(10)

,type,text,spam
0,ham,"[go, jurong, point,, crazy.., avail, onli, bug...",0
1,ham,"[ok, lar..., joke, wif, u, oni...]",0
2,spam,"[free, entri, 2, wkli, comp, win, fa, cup, fin...",1
3,ham,"[u, dun, say, earli, hor..., u, c, alreadi, sa...",0
4,ham,"[nah, think, goe, usf,, live, around, though]",0
5,spam,"[freemsg, hey, darl, 3, week, word, back!, i'd...",1
6,ham,"[even, brother, like, speak, me., treat, like,...",0
7,ham,"[per, request, mell, mell, (oru, minnaminungin...",0
8,spam,"[winner!!, valu, network, custom, select, rece...",1
9,spam,"[mobil, 11, month, more?, u, r, entitl, updat,...",1


# 6) Transformation of text data into TDF/TF-IDF Vectors

In [29]:
tfidf = TfidfVectorizer()
y=dt.spam.values
dt['text'] = dt['text'].apply(lambda x: ' '.join(x))
x=tfidf.fit_transform(dt['text'])

In [31]:
x_train, x_text, y_train, y_text = train_test_split(x ,y ,random_state=1,test_size=0.2,shuffle=False)

# 7) Classification using Logistic Regression

In [32]:
clf=LogisticRegression()
clf.fit(x_train, y_train)
y_pred=clf.predict(x_text)

acc_log = accuracy_score(y_pred, y_text)*100
print("accuray: ", acc_log)

accuray:  87.5


# 8) Classification using LinearSVC Accuracy

In [33]:
linear_svc = LinearSVC(random_state=0)
linear_svc.fit(x_train, y_train)
y_pred = linear_svc.predict(x_text)
acc_linear_svc = accuracy_score(y_pred, y_text)*100
print("accuracy", acc_linear_svc)

accuracy 87.5
